Coleta de Noticias

In [80]:
## Fontes de Notícias

fonte_g1 = "https://g1.globo.com/economia/dolar/"
fonte_cnn = "https://www.cnnbrasil.com.br/tudo-sobre/dolar/"
fonte_infomoney = "https://www.infomoney.com.br/tudo-sobre/dolar/"

In [18]:
## Importando Bibliotecas para Web Scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import datetime
import time
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [6]:
## Extraindo Notícias do G1 com o Selenium
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(fonte_g1)
time.sleep(5)  # Aguarda o carregamento da página
driver.find_element(By.CSS_SELECTOR, "svg.fc-cancel-icon-svg").click() # Fecha o pop-up de ADS

# Scroll para carregar mais notícias
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
html_g1 = driver.page_source


driver.quit()


In [ ]:

soup = BeautifulSoup(html_g1, "html.parser")

# Tratando os dados do G1
url_imagem = [img["src"] for img in soup.select("img.bstn-fd-picture-image")]
titulos = [t.get_text(strip=True) for t in soup.select("div.feed-post-body-title a")]
subtitulos = [s.get_text(strip=True) for s in soup.select("div.feed-post-body-resumo")]
links = [a["href"] for a in soup.select("a.feed-post-link")]
data_publicacao = [
    f"{link.split('/')[7]}/{link.split('/')[6]}/{link.split('/')[5]}"
    for link in links
]

# Salvando os dados em um DataFrame com a data de extração
data_extracao = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
df_g1 = pd.DataFrame({
    "urlImagem": url_imagem,
    "dataPublicacao": data_publicacao,
    "titulo": titulos,
    "subtitulo": subtitulos,
    "link": links,
    "fonte": "G1",
    "dataExtracao": data_extracao
})

df_noticias = df_g1.copy()

In [ ]:
## Extraindo Notícias do CNN 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(fonte_cnn)
time.sleep(5)  # Aguarda o carregamento da página

wait = WebDriverWait(driver, 10)
botao_ok = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-agree button")))

# Clica no botão
botao_ok.click()


for i in range(2):  # Navega por 2 páginas
    time.sleep(5)
    html_cnn = driver.page_source
    soup = BeautifulSoup(html_cnn, "html.parser")
    container = soup.select_one('ul[data-section="article_list"]')
    

    # Printa a pagina percorrida
    url_imagem= []
    if container:
        url_imagem = [img["src"] for img in container.select("img") if img.get("src")]
    titulos = [t.get_text(strip=True) for t in soup.select("div.flex.flex-col.gap-4 h2")]
    links = [a["href"] for a in soup.select("div.flex.flex-col.gap-4 a")]
    links = [item for item in links if item != 'https://www.cnnbrasil.com.br/ao-vivo/']
    data_publicacao = []
    for tag in soup.select("time"):
        raw = tag.get("datetime")
        if raw:
            data_publicacao.append(datetime.fromisoformat(raw).strftime("%d/%m/%Y"))

    data_extracao = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_cnn = pd.DataFrame({
        "urlImagem": url_imagem,
        "dataPublicacao": data_publicacao,
        "titulo": titulos,
        "subtitulo": "",
        "link": links,
        "fonte": "CNN",
        "dataExtracao": data_extracao
    })

    df_noticias = pd.concat([df_noticias, df_cnn], ignore_index=True)

    driver.find_element(By.CSS_SELECTOR, "a[aria-label='Ir para próxima página']").click()    
    
    time.sleep(5)

driver.quit()


In [9]:
display(df_noticias)

,dataPublicacao,titulo,subtitulo,link,fonte,dataExtracao
0,23/10/2025,Dólar recua com tensões globais no radar; Ibov...,"Na véspera, a moeda norte-americana avançou 0,...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
1,22/10/2025,"Dólar tem leve alta e fecha a R$ 5,39 com expe...","A moeda norte-americana avançou 0,13%, cotada ...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
2,21/10/2025,"Dólar sobe e fecha em R$ 5,39, com expectativa...","A moeda norte-americana avançou 0,36%, cotada ...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
3,20/10/2025,"Dólar cai e fecha em R$ 5,37, com atenção ao m...","A moeda americana caiu 0,63%, cotada a R$ 5,37...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
4,17/10/2025,"Dólar cai a R$ 5,40 de olho em negociação do t...","A moeda americana caiu 0,69%, cotada a R$ 5,40...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
5,16/10/2025,"Dólar cai e fecha a R$ 5,44 com foco nas negoc...","A moeda americana caiu 0,36%, cotada a R$ 5,44...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
6,15/10/2025,"Dólar tem leve queda e fecha a R$ 5,46 de olho...","A moeda americana recuou 0,13%, cotada a R$ 5,...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
7,14/10/2025,"Dólar tem leve alta e fecha a R$ 5,46 com tens...","A moeda americana avançou 0,14%, cotada a R$ 5...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
8,13/10/2025,"Dólar cai e fecha R$ 5,46 com tom mais ameno d...","A moeda americana recuou 0,75%, cotada a R$ 5,...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
9,10/10/2025,"Dólar dispara e fecha a R$ 5,50 após ameaça de...","A moeda americana avançou 2,38%, cotada a R$ 5...",https://g1.globo.com/economia/noticia/2025/10/...,G1,23/10/2025 15:00:53
